In [2]:
import json
import regex as re

In [3]:
with open('../texts.json', 'r') as data:
    texts = json.load(data)

In [4]:
#this can be a good start, create a long text with all the dataset formated as author, title and text.
full_text = "" # # create a big string that joins all texts with format author, title, text
poems = "" # create a big string that joins the poems with format author, title, text

for text in texts:
    if text['text_type']=='poem':
        poems += text['author'] + '\n' + text['title'] + text['text'] + '<end>' #added end token to know when to stop sampling
    full_text += text['author'] + '\n' + text['title'] + text['text']


In [5]:
characters = sorted(list(set(full_text)))
vocab = {}
vocab[0] = '<end>'
for a,b in enumerate(characters, start=1):
    vocab[a] = b
print(vocab)

{0: '<end>', 1: '\n', 2: '\r', 3: ' ', 4: '!', 5: '"', 6: '$', 7: '%', 8: '&', 9: "'", 10: '(', 11: ')', 12: '*', 13: '+', 14: ',', 15: '-', 16: '.', 17: '/', 18: '0', 19: '1', 20: '2', 21: '3', 22: '4', 23: '5', 24: '6', 25: '7', 26: '8', 27: '9', 28: ':', 29: ';', 30: '<', 31: '=', 32: '>', 33: '?', 34: 'A', 35: 'B', 36: 'C', 37: 'D', 38: 'E', 39: 'F', 40: 'G', 41: 'H', 42: 'I', 43: 'J', 44: 'K', 45: 'L', 46: 'M', 47: 'N', 48: 'O', 49: 'P', 50: 'Q', 51: 'R', 52: 'S', 53: 'T', 54: 'U', 55: 'V', 56: 'W', 57: 'X', 58: 'Y', 59: 'Z', 60: '[', 61: '\\', 62: ']', 63: '_', 64: 'a', 65: 'b', 66: 'c', 67: 'd', 68: 'e', 69: 'f', 70: 'g', 71: 'h', 72: 'i', 73: 'j', 74: 'k', 75: 'l', 76: 'm', 77: 'n', 78: 'o', 79: 'p', 80: 'q', 81: 'r', 82: 's', 83: 't', 84: 'u', 85: 'v', 86: 'w', 87: 'x', 88: 'y', 89: 'z', 90: '|', 91: '}', 92: '\xa0', 93: '¤', 94: '¥', 95: '§', 96: 'ª', 97: '«', 98: '\xad', 99: '°', 100: '´', 101: 'º', 102: '»', 103: '¼', 104: 'À', 105: 'Á', 106: 'Â', 107: 'Ã', 108: 'Å', 109: '

In [6]:
print(poems)

Ricardo Reis
Diana através dos ramos

Diana através dos ramos
Espreita a vinda de Endymion
Endymion que nunca vem,
Endymion, Endymion,
Lá longe na floresta…
E a sua voz chamando
Exclama através dos ramos
Endymion, Endymion…

Assim choram os deuses…
<end>Ricardo Reis
Aqui, sem outro Apolo do que Apolo,

Aqui, sem outro Apolo do que Apolo,
Sem um suspiro abandonemos Cristo
        E a febre de buscarmos
        Um deus dos dualismos.

E longe da cristã sensualidade
Que a casta calma da beleza antiga
        Nos restitua o antigo
        Sentimento da vida.
<end>Ricardo Reis
Em Ceres anoitece.

Em Ceres anoitece.
Nos píncaros ainda
        Faz luz.

Sinto-me tão grande
Nesta hora solene
        E vã

Que, assim como há deuses
Dos campos, das flores
        Das searas,


Agora eu quisera
Que um deus existisse
        De mim.
<end>Ricardo Reis
Não a ti, mas aos teus, odeio, Cristo.

Não a ti, mas aos teus, odeio, Cristo.
Tu não és mais que um deus a mais no eterno
        Pantéon que presid

In [7]:
def get_pair_frequency(text): #checks for the number of token pairs in the text. Returns dictionary with pair: # of occurrences
    pair_frequency = {}
    for pair in zip(text, text[1:]):
        pair_frequency[pair] = pair_frequency.get(pair, 0) + 1 #tries to get pair, if it doesn't exist assigns 0 and sums 1
    return pair_frequency

In [8]:
pair_frequency = get_pair_frequency(poems)
top_pair, _ = sorted(pair_frequency.items(), key=lambda x:x[1], reverse=True)[0] #sorts dictionary by highest value
def merge_pair(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i< len(ids)-1 and pair[0] == ids[i] and pair[1] == ids[i+1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids
print(merge_pair('abababola', 'ab','!'))
#pair_frequency


['!', '!', '!', 'o', 'l', 'a']


In [9]:
vocab = {idx:bytes([idx]) for idx in range(256)}
merges = {}
def train(vocab_size, text):
    ids = text.encode('utf-8')
    for i in range(vocab_size-len(vocab)):
        pair_frequency = get_pair_frequency(ids)
        top_pair, _ = sorted(pair_frequency.items(), key=lambda x:x[1], reverse=True)[0] #sorts dictionary by highest value
        idx = len(vocab)
        print(f'Merging {top_pair} -> {idx}')
        merges[idx] = top_pair
        ids = merge_pair(ids, top_pair, idx)
        vocab[idx] = top_pair


In [10]:
train(vocab_size=270,text=poems)

Merging (101, 32) -> 256
Merging (111, 32) -> 257
Merging (97, 32) -> 258
Merging (115, 32) -> 259
Merging (194, 160) -> 260
Merging (260, 260) -> 261
Merging (101, 114) -> 262
Merging (44, 32) -> 263
Merging (101, 110) -> 264
Merging (113, 117) -> 265
Merging (109, 32) -> 266
Merging (101, 115) -> 267
Merging (111, 114) -> 268
Merging (97, 114) -> 269


In [67]:
print(vocab[111], vocab[32])

b'o' b' '


In [16]:
""" Important considerations for the tokenizer.
Must not create tokens across sentences/ verses/ paragraphs. I think should be case insensitive.
Shouldn't merge words with punctuation. Should be able to merge punctuation together such as the aesthetical spaces in the beggining of some verses.
=> I should create a regex pattern.
I think I should add at least a special token to separate accross texts for the transformer training.
 """

" Important considerations for the tokenizer.\nMust not create tokens across sentences/ verses/ paragraphs. I think should be case insensitive.\nShouldn't merge words with punctuation. Should be able to merge punctuation together such as the aesthetical spaces in the beggining of some verses.\n=> I should create a regex pattern.\nI think I should add at least a special token to separate accross texts for the transformer training.\n "

In [21]:
paragraphs = []
for t in text_list:
    t = t.lower() #lowercse for case insensitivity
    paragraphs.append(t)
    #paragraphs.append(t.split('\n')) #split by verse/paragraph. I don't want to create tokens across paragraphs

In [36]:
paragraphs

['\ndiana através dos ramos\nespreita a vinda de endymion\nendymion que nunca vem,\nendymion, endymion,\nlá longe na floresta…\ne a sua voz chamando\nexclama através dos ramos\nendymion, endymion…\n\nassim choram os deuses…\n',
 '\na reforma do calendário e as suas consequências comerciais\n\na comissão especial de inquérito à reforma do calendário, estabelecida pela sociedade das nações em 1924, acaba de publicar o seu relatório.\na comissão era composta do prof. van eysinga, da universidade de leyden (presidente); rev. t.e.r. philips, secretário da real sociedade astronómica de londres, nomeado pelo arcebispo de canterbury, chefe da igreja inglesa; rev. p. gianfranceschi, presidente da academia dei nuovi lincei, nomeado pela santa sé; prof. eginitis, director do observatório de atenas, representando o patriarca ecuménico de constantinopla; o sr. bigourden, antigo presidente da comissão de calendário da união astronómica internacional; e o sr. willis h. booth, antigo presidente da câm

Regex Patterns

In [38]:
pattern = re.compile(
    r"""
    [A-Za-zÀ-ÿ0-9]+(?:'[A-Za-zÀ-ÿ0-9]+)?   # Words, including Portuguese accents and contractions
    | [.!?]+                               # End-of-sentence punctuation marks
    | ,                                    # Comma as a separate token
    | [\(\)\"\“\”\']                       # Brackets or quotation marks as separate tokens
    | [\s\xa0]{2,}                         # Multiple spaces
    | [\n]                               # Newline (to separate verses/paragraphs)
    """, 
    re.VERBOSE
)

patterngpt=re.compile(r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""")